In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install dgl-cu113 dglgo -f https://data.dgl.ai/wheels/repo.html
#!pip install torchviz
!pip install wandb
!wandb login

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.dgl.ai/wheels/repo.html
     |████████████████████████████████| 220.6 MB 35 kB/s 
     |████████████████████████████████| 77 kB 4.9 MB/s 
     |████████████████████████████████| 281 kB 68.6 MB/s 
     |████████████████████████████████| 596 kB 64.9 MB/s 
     |████████████████████████████████| 45 kB 3.6 MB/s 
     |████████████████████████████████| 109 kB 84.3 MB/s 
     |████████████████████████████████| 103 kB 70.8 MB/s 
     |████████████████████████████████| 51 kB 509 kB/s 
     |████████████████████████████████| 11.1 MB 63.3 MB/s 
     |████████████████████████████████| 42 kB 908 kB/s 
     |████████████████████████████████| 3.1 MB 62.8 MB/s 
     |████████████████████████████████| 546 kB 24.3 MB/s 
     |████████████████████████████████| 90 kB 7.0 MB/s 
     |████████████████████████████████| 100 kB 9.6 MB/s 
     |████████████

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 32.4 MB/s 
     |████████████████████████████████| 181 kB 56.2 MB/s 
     |████████████████████████████████| 145 kB 77.2 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=ee1fd293ea15fdb49a880fe0373640d8513155e863eaaf0ba454452c2bdc5d58
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [2]:
import sys
sys.path.insert(0,'/content/drive/My Drive/SIG-VAE-GIN-and-fMRI/SIG-VAE-GIN-and-fMRI')

In [2]:
%ls
%load_ext autoreload
%autoreload 2/

drive/  sample_data/


In [3]:
%cd drive
%cd MyDrive
%cd SIG-VAE-GIN-and-fMRI
%cd SIG-VAE-GIN-and-fMRI
from sigvaemodel_transformer import *
from loss import *

/content/drive
/content/drive/MyDrive
/content/drive/MyDrive/SIG-VAE-GIN-and-fMRI
/content/drive/MyDrive/SIG-VAE-GIN-and-fMRI/SIG-VAE-GIN-and-fMRI


DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [4]:
%reload_ext autoreload
from sigvaemodel_transformer import *
from loss import loss as loss_function
#from graph_transformer_edge_layer import *
from graph_transformer_layer import *
from mlp_readout_layer import *
#from loss import get_rec

In [6]:
import numpy as np
import torch
import torch.nn as nn
import dgl.data
import copy
import networkx as nx
import scipy.sparse as sp
from sklearn.metrics import average_precision_score, roc_curve, precision_recall_curve, roc_auc_score
import sigvaemodel_transformer
import loss
import time
import wandb
import torch.optim as optim
import gc
from matplotlib import pyplot
#from torchviz import make_dot

CoraDataset=dgl.data.CoraGraphDataset(reverse_edge=False)
graph=CoraDataset[0]
u, v=graph.edges()
print(len(u))
featureMatrix=graph.ndata['feat']
featureMatrix=featureMatrix.view(1, featureMatrix.shape[0], featureMatrix.shape[1])

print(featureMatrix.size())
print(featureMatrix)
graph.to('cuda')
config=dict(
    learning_rate=0.005,
    architecture="GIN",
    dataset="Cora",
    noise_dim=64,
    u_dim = 32,
    latent_dim = 32,
    lap_pos_enc_dim=4,
    warming_up=200,
    K=15,
    J=15,
    epoch=2000
)


Extracting file to /root/.dgl/cora_v2
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
10556
torch.Size([1, 2708, 1433])
tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0526, 0.0000]]])


In [7]:
def edges_to_lists(headMatrix, tailMatrix):
    results=[]
    assert len(headMatrix)==len(tailMatrix)
    hlength=len(headMatrix)
    for i in range(hlength):
        results.append([headMatrix[i], tailMatrix[i]])
    return results

In [8]:
def laplacian_positional_encoding(g, pos_enc_dim):
    """
        Graph positional encoding v/ Laplacian eigenvectors
    """

    # Laplacian
    A = g.adjacency_matrix_scipy(return_edge_ids=False).astype(float)
    N = sp.diags(dgl.backend.asnumpy(g.in_degrees()).clip(1) ** -0.5, dtype=float)
    L = sp.eye(g.number_of_nodes()) - N * A * N

    # Eigenvectors with scipy
    #EigVal, EigVec = sp.linalg.eigs(L, k=pos_enc_dim+1, which='SR')
    EigVal, EigVec = sp.linalg.eigs(L, k=pos_enc_dim+1, which='SR', tol=1e-2) # for 40 PEs
    EigVec = EigVec[:, EigVal.argsort()] # increasing order
    g.ndata['lap_pos_enc'] = torch.from_numpy(EigVec[:,1:pos_enc_dim+1]).float() 

    return g

In [9]:
def preprocess(graph, pos_enc_dim):
    N=graph.num_nodes()
    numE=graph.number_of_edges()
    halfnumE=int(numE/2)
    
    u, v=graph.edges()
    #graph.add_edges(v, u, graph.edata)
    #u, v=graph.edges()
    #numEtimes2=graph.number_of_edges()
    #print(numEtimes2)
    idx=np.arange(halfnumE)
    testidx=np.random.permutation(idx)
    testSize=int(len(idx)/10)
    validation_size=int(len(idx)/20)
    train_size=numE-testSize-validation_size
    '''
    adj1=sp.coo_matrix((np.ones(len(u)), (u.detach().cpu().numpy(), v.detach().cpu().numpy())))
    adj2=sp.coo_matrix((np.ones(len(u)), (v.detach().cpu().numpy(), u.detach().cpu().numpy())))
    adj=adj1+adj2
    adj=torch.Tensor(adj.todense())
    print(adj)
    adj[adj==2]=1
    '''
    adj=sp.coo_matrix((np.ones(len(u)), (u.detach().cpu().numpy(), v.detach().cpu().numpy())))
    adj=torch.Tensor(adj.todense())
    adj_upper=torch.triu(adj, diagonal=1)
    adj_lower=torch.tril(adj, diagonal=-1)

    uUpper, vUpper=torch.where(adj_upper!=0)
    print("^^^^^^^^^^^^^^^^6666")
    print(uUpper.shape)
    print(vUpper.shape)
    test_pos_u, test_pos_v=uUpper[testidx[:testSize]], vUpper[testidx[:testSize]]
    train_pos_u, train_pos_v = uUpper[testidx[testSize:halfnumE-validation_size]], vUpper[testidx[testSize:halfnumE-validation_size]]
    validation_pos_u, validation_pos_v=uUpper[testidx[halfnumE-validation_size:]], vUpper[testidx[halfnumE-validation_size:]]
    
    #print("&&&&&&&&&&&&&&&&&&&&&")
    #print(adj.shape)
    #print(torch.max(adj))
    adj_neg_upper=torch.triu(1-adj_upper, diagonal=1)
    neg_u, neg_v=torch.where(adj_neg_upper!=0)
    #print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
    #print(len(neg_u))
    #print(len(neg_v))
    neg_testidx=np.random.choice(len(neg_u), halfnumE)
    test_neg_u, test_neg_v=neg_u[neg_testidx[:testSize]], neg_v[neg_testidx[:testSize]]
    train_neg_u, train_neg_v=neg_u[neg_testidx[testSize:halfnumE-validation_size]], neg_v[neg_testidx[testSize:halfnumE-validation_size]]
    validation_neg_u, validation_neg_v=neg_u[neg_testidx[halfnumE-validation_size:]], neg_v[neg_testidx[halfnumE-validation_size:]]
    #print(test_neg_u)
    #print(test_neg_v)
    
    adj_train1=sp.coo_matrix((np.ones(len(train_pos_u)), (train_pos_u.detach().cpu().numpy(), train_pos_v.detach().cpu().numpy())),shape=(N, N))
    adj_train2=sp.coo_matrix((np.ones(len(train_pos_u)), (train_pos_v.detach().cpu().numpy(), train_pos_u.detach().cpu().numpy())),shape=(N, N))
    adj_train=adj_train1+adj_train2
    #adj_train[adj_train==2]=1
    #adj_train=adj_train-sp.dia_matrix((adj_train.diagonal()[np.newaxis, :], [0]), shape=adj_train.shape)
    adj_train=torch.Tensor(adj_train.todense())
    print(torch.max(adj_train))
    
    adj_test1=sp.coo_matrix((np.ones(len(test_pos_u)), (test_pos_u.detach().cpu().numpy(), test_pos_v.detach().cpu().numpy())),shape=(N,N))
    adj_test2=sp.coo_matrix((np.ones(len(test_pos_u)), (test_pos_v.detach().cpu().numpy(), test_pos_u.detach().cpu().numpy())),shape=(N,N))
    adj_test=adj_test1+adj_test2
    #adj_test[adj_test==2]=1
    #adj_test=adj_test-sp.dia_matrix((adj_test.diagonal()[np.newaxis, :], [0]), shape=adj_test.shape)
    adj_test=torch.Tensor(adj_test.todense())
    print(torch.max(adj_test))

    adj_validation1=sp.coo_matrix((np.ones(len(validation_pos_u)), (validation_pos_u.detach().cpu().numpy(), validation_pos_v.detach().cpu().numpy())),shape=(N,N))
    adj_validation2=sp.coo_matrix((np.ones(len(validation_pos_u)), (validation_pos_v.detach().cpu().numpy(), validation_pos_u.detach().cpu().numpy())),shape=(N,N))
    adj_validation=adj_validation1+adj_validation2
    #adj_validation[adj_validation==2]=1
    #adj_validation=adj_validation-sp.dia_matrix((adj_validation.diagonal()[np.newaxis, :], [0]), shape=adj_validation.shape)
    adj_validation=torch.Tensor(adj_validation.todense())
    print(torch.max(adj_validation))


    adj_train_neg1=sp.coo_matrix((np.ones(len(train_neg_u)), (train_neg_u, train_neg_v)),shape=(N, N))
    adj_train_neg2=sp.coo_matrix((np.ones(len(train_neg_u)), (train_neg_v, train_neg_u)),shape=(N, N))
    adj_train_neg=adj_train_neg1+adj_train_neg2
    
    #adj_train=adj_train-sp.dia_matrix((adj_train.diagonal()[np.newaxis, :], [0]), shape=adj_train.shape)
    adj_train_neg=torch.Tensor(adj_train_neg.todense())
    adj_train_neg[adj_train_neg==2]=1
    print(torch.max(adj_train_neg))
    
    adj_test_neg1=sp.coo_matrix((np.ones(len(test_neg_u)), (test_neg_u, test_neg_v)),shape=(N,N))
    adj_test_neg2=sp.coo_matrix((np.ones(len(test_neg_u)), (test_neg_v, test_neg_u)),shape=(N,N))
    adj_test_neg=adj_test_neg1+adj_test_neg2
    adj_test[adj_test==2]=1
    #adj_test=adj_test-sp.dia_matrix((adj_test.diagonal()[np.newaxis, :], [0]), shape=adj_test.shape)
    adj_test_neg=torch.Tensor(adj_test_neg.todense())
    print(torch.max(adj_test_neg))

    adj_validation_neg1=sp.coo_matrix((np.ones(len(validation_neg_u)), (validation_neg_u, validation_pos_v)),shape=(N,N))
    adj_validation_neg2=sp.coo_matrix((np.ones(len(validation_neg_u)), (validation_neg_v, validation_pos_u)),shape=(N,N))
    adj_validation_neg=adj_validation_neg1+adj_validation_neg2
    adj_validation[adj_validation==2]=1
    #adj_validation=adj_validation-sp.dia_matrix((adj_validation.diagonal()[np.newaxis, :], [0]), shape=adj_validation.shape)
    adj_validation_neg=torch.Tensor(adj_validation_neg.todense())
    print(torch.max(adj_validation_neg))


    train_u, train_v=torch.where(adj_train!=0)
    train_graph=dgl.graph((torch.hstack((train_u, train_v)), torch.hstack((train_v, train_u))), num_nodes=N)
    
    #train_graph=dgl.remove_edges(graph, np.hstack((testidx[:testSize],testidx[numE-validation_size:], (testidx[:testSize]+numE), (testidx[numE-validation_size:]+numE))))
    #train_graph=dgl.remove_edges(train_graph_temp, np.hstack((testidx[:testSize], testidx[numE-validation_size:]+numE)))
    test_u, test_v=torch.where(adj_test!=0)
    #test_graph=dgl.graph((torch.hstack((test_u, test_v)), torch.hstack((test_v, test_u))))
    #test_graph=dgl.remove_edges(graph, np.hstack((testidx[testSize:], (testidx[testSize:]+numE))))
    #test_graph=dgl.remove_edges(test_graph_temp, testidx[testSize:]+numE)
    validation_u, validation_v=torch.where(adj_validation!=0)
    #validation_graph=dgl.graph((torch.hstack((validation_u, validation_v)), torch.hstack((validation_v, validation_u))))
    #validation_graph=dgl.remove_edges(graph, np.hstack((testidx[:numE-validation_size], (testidx[:numE-validation_size]+numE))))
    #validation_graph=dgl.remove_edges(validation_graph_temp ,testidx[:numE-validation_size]+numE)
    
    train_neg_u, train_neg_v=torch.where(adj_train_neg!=0)
    test_neg_u, test_neg_v=torch.where(adj_test_neg!=0)
    validation_neg_u, validation_neg_v=torch.where(adj_validation_neg!=0)
    '''
    train_pos=edges_to_lists(train_pos_u, train_pos_v)
    test_pos=edges_to_lists(test_pos_u, test_pos_v)
    test_neg=edges_to_lists(test_neg_u, test_neg_v)
    train_neg=edges_to_lists(train_neg_u, train_neg_v)
    validation_pos=edges_to_lists(validation_pos_u, validation_pos_v)
    validation_neg=edges_to_lists(validation_neg_u, validation_neg_v)
    '''
    '''
    train_pos=edges_to_lists(train_pos_u, train_pos_v)+edges_to_lists(train_pos_v, train_pos_u)
    test_pos=edges_to_lists(test_pos_u, test_pos_v)+edges_to_lists(test_pos_v, test_pos_u)
    test_neg=edges_to_lists(test_neg_u, test_neg_v)+edges_to_lists(test_neg_v, test_neg_u)
    train_neg=edges_to_lists(train_neg_u, train_neg_v)+edges_to_lists(train_neg_v, train_neg_u)
    validation_pos=edges_to_lists(validation_pos_u, validation_pos_v)+edges_to_lists(validation_pos_v, validation_pos_u)
    validation_neg=edges_to_lists(validation_neg_u, validation_neg_v)+edges_to_lists(validation_neg_v, validation_neg_u)
    '''
    train_pos=edges_to_lists(train_u, train_v)
    test_pos=edges_to_lists(test_u, test_v)
    validation_pos=edges_to_lists(validation_u, validation_v)
    train_neg=edges_to_lists(train_neg_u, train_neg_v)
    test_neg=edges_to_lists(test_neg_u, test_neg_v)
    validation_neg=edges_to_lists(validation_neg_u, validation_neg_v)

    print(len(validation_pos))
    print(len(validation_neg))
    featureMatrix=graph.ndata['feat']
    print("****************")
    print(featureMatrix.shape[0])
    featureMatrix=featureMatrix.view(1, featureMatrix.shape[0], featureMatrix.shape[1]).to('cuda')

    train_graph=laplacian_positional_encoding(train_graph, pos_enc_dim)
    #test_graph=laplacian_positional_encoding(test_graph, pos_enc_dim)
    #validation_graph=laplacian_positional_encoding(validation_graph, pos_enc_dim)

    return graph, adj, adj_train, adj_validation, adj_test, featureMatrix, train_graph, train_pos, validation_pos, test_pos, train_neg, validation_neg, test_neg


In [10]:
def step(model, adj, adj_label, feat, input_graph, batch_graph, epoch, K, J, weight, norm, dropout, optimizer, scheduler, device):
  #global loss
    if optimizer is None:
        with torch.no_grad():
          model.eval()
    else:
        model.train()
        optimizer.zero_grad()
    WU=torch.min(torch.Tensor([epoch/200., 1.]))
    
    #generated_prob, mu, sigma, latent_representation, Z, epsilon=model.forward(adj.to(device), feat.to(device), input_graph)
    generated_prob, mu, sigma, latent_representation, Z, epsilon, rk=model.forward(adj.to(device), feat.to(device), input_graph, batch_graph)
    loss=loss_function(generated_prob, adj_label, mu, sigma, Z, epsilon, latent_representation, K, J, WU, norm, weight, dropout, device=device)
    
    #loss_rec, loss_prior, loss_post=loss_function(generated_prob, adj_label, mu, sigma, Z, epsilon, latent_representation, K, J, norm, weight, dropout, device=device)
    #reg=(loss_post-loss_prior) / (adj.size(0)**2)
    #loss=loss_rec+reg
    
    #print(generated_prob.shape)
    #print(adj.shape)
    #loss=loss_rec+loss_prior+loss_post
    
    #WarmingUP=np.min([epoch/300,1])
    #reg=(loss_post-loss_prior) * WarmingUP / (adj.size(0)**2)
    #loss=loss_rec+WarmingUP*reg
    
    
    if optimizer is not None:
        #make_dot(loss,params=dict(model.named_parameters()))
        loss.backward()
        optimizer.step()
        if scheduler is not None:
          scheduler.step()
    del mu
    del sigma
    del epsilon
    return latent_representation, generated_prob, loss, Z, rk

In [11]:
def get_auc(embedding, adj_orig, rk, pos_array, neg_array, decoder_type):
    
    def dc(x):
      if decoder_type=="ip":
        return 1/(1+torch.exp(-x))
      elif decoder_type=="bp":
        return 1-torch.exp(-torch.exp(x))
    
    J=embedding.shape[0]
    rk_expand=torch.tile(torch.diag(rk), (J, 1, 1))
    X=torch.transpose(embedding, 1, 2)
    X=torch.bmm(rk_expand, X)
    X=torch.bmm(embedding, X)
    X=torch.clamp(X, min=float('-inf'), max=10)
    adj_rec=1-torch.exp(-torch.exp(X))
    #print("@@@@@@@@@@@@@@@@@@@@")
    #print(adj_rec)
    '''
    adj_rec=torch.zeros(J, embedding.shape[1], embedding.shape[1])
    for i, emb in enumerate(embedding):
      tmp = torch.matmul(torch.diag(rk), emb.t())
      adj_rec_sub = 1 - torch.exp(- torch.exp(torch.matmul(emb, tmp)))
      adj_rec[i]=adj_rec_sub
    '''
    preds_pos=torch.zeros(J, pos_array.shape[1])
    for i, e in enumerate(pos_array.t()):
      preds_pos[:,i]=adj_rec[:,e[0], e[1]]
    
    preds_neg=torch.zeros(J, neg_array.shape[1])
    for i, e in enumerate(neg_array.t()):
      preds_neg[:,i]=adj_rec[:, e[0], e[1]]

    #preds_pos=adj_rec[:, pos_array[0], pos_array[1]]
    #preds_neg=adj_rec[:, neg_array[0], neg_array[1]]
    '''
    emb_pos_head=embedding[:, pos_array[0], :]
    emb_pos_tail=embedding[:, pos_array[1], :]
    preds_pos=dc(torch.einsum('abc, abc->ab', emb_pos_head, emb_pos_tail))
    emb_neg_head=embedding[:, neg_array[0], :]
    emb_neg_tail=embedding[:, neg_array[1], :]
    preds_neg=dc(torch.einsum('abc, abc->ab', emb_neg_head, emb_neg_tail))
    #preds_neg=torch.einsum('abc, abc->ab', emb_neg_head, emb_neg_tail)
    '''

    preds=torch.hstack([preds_pos, preds_neg]).detach().cpu().numpy()
    labels=torch.hstack([torch.ones(preds_pos.shape[-1]), torch.zeros(preds_neg.shape[-1])]).detach().cpu().numpy()
    #print(preds[0])
    #print(preds[1])
    roc=torch.Tensor([roc_auc_score(labels, pred.flatten()) for pred in np.vsplit(preds, embedding.shape[0])]).mean()
    avg_precision=torch.Tensor([average_precision_score(labels, pred.flatten()) for pred in np.vsplit(preds, embedding.shape[0])]).mean()
    '''
    fpr_accumulate=0
    tpr_accumulate=0
    recall_accumulate=0
    precision_accumulate=0
    for pred in np.vsplit(preds,J):
      fpr, tpr, _=roc_curve(labels, pred.flatten())
      precision, recall, _=precision_recall_curve(labels, pred.flatten())
      fpr_accumulate+=fpr.mean()
      tpr_accumulate+=tpr.mean()
      recall_accumulate+=recall.mean()
      precision_accumulate+=precision.mean()
    fpr_mean=fpr_accumulate/J
    tpr_mean=tpr_accumulate/J
    recall_mean=recall_accumulate/J
    precision_mean=precision_accumulate/J
    '''
    #precision, recall=torch.Tensor([precision_recall_curve(labels, pred.flatten()) for pred in np.vsplit(preds, embedding.shape[0])])
    #accuracy=torch.Tensor([accuracy_score(labels, pred.flatten()) for pred in np.vsplit(preds, embedding.shape[0])]).mean()
    #precision=torch.Tensor([precision_score(labels, pred) for pred in np.vsplit(preds, embedding.shape[0])]).mean()
    #recall=torch.Tensor([recall_score(labels, pred) for pred in np.vsplit(preds, embedding.shape[0])]).mean()
    #precision=precision.mean()
    #recall=recall.mean()
    #fpr=fpr.mean()
    del adj_rec
    del preds_pos
    del preds_neg
    del preds
    del labels

    return roc, avg_precision #, precision_mean, recall_mean, fpr_mean, tpr_mean

In [ ]:
if torch.cuda.is_available():
  device='cuda'
else:
  device='cpu'

import wandb

wandb.config = {
  "learning_rate": 0.005,
  "epochs": 2000,
  "u_dim" : 32,
  "latent_dim" : 32,
  "batch_size": 30,
  "K" : 15,
  "J" : 15, 
  "WU" : 200,
  "noise_dim" : 64,
  "pos_enc_dim" : 4
}

wandb.init(project="SIG-VAE-GIN-and-fMRI", entity="ytrewq271828", config=config)

laplacian_positional_encoding_dim=4
graph_processed, adj, adj_train, adj_validation, adj_test, featureMatrix, train_graph, train_pos, validation_pos, test_pos, train_neg, validation_neg, test_neg=preprocess(graph, laplacian_positional_encoding_dim)
adj_label=adj_train+torch.eye(adj_train.shape[0])
adj_label=torch.FloatTensor(adj_label)

#print((adj.transpose(0,1)==adj).all())
##print((adj_train.transpose(0,1)==adj_train).all())
#print((adj_test.transpose(0,1)==adj_test).all())
#print((adj_validation.transpose(0,1)==adj_validation).all())
adj=torch.Tensor(adj).to(device)
adj_test=torch.Tensor(adj_test).to(device)
featureMatrix=torch.cuda.FloatTensor(featureMatrix)
adj_train=torch.Tensor(adj_train).to(device)
train_graph=train_graph.to(device)
#test_graph=test_graph.to(device)
#validation_graph=validation_graph.to(device)
graph_processed=graph_processed.to(device)
N=adj.shape[0]
weight=torch.tensor([float(N * N - adj.sum()) / adj.sum()]).to(device)
norm=N*N/float((N*N-adj.sum())*2)

pos_array_train=torch.cuda.LongTensor(train_pos).t() #(2, #positive edges)
neg_array_train=torch.cuda.LongTensor(train_neg).t() #(2, #negative edges)
pos_array_test=torch.cuda.LongTensor(test_pos).t()
neg_array_test=torch.cuda.LongTensor(test_neg).t()
pos_array_validation=torch.cuda.LongTensor(validation_pos).t()
neg_array_validation=torch.cuda.LongTensor(validation_neg).t()
#print(torch.count_nonzero(adj_train))
#print(torch.count_nonzero(adj_test))
#print(adj_test.shape)
#print(pos_array_train)?
print(pos_array_validation.shape)
print(neg_array_validation.shape)
print("%%%%%%%%%%%%%%%%%%%%%55")
print(train_graph.number_of_nodes())
torch.autograd.set_detect_anomaly(False)

model=SIGVAE_GIN_Transformer(Lu=1, 
                Lmu=1, 
                Lsigma=1, 
                input_dim=featureMatrix.shape[2], 
                output_dim_u=[32], 
                output_dim_mu=[32], 
                output_dim_sigma=[32],
                K=15,
                J=15,
                lap_pos_enc_dim=4,
                device=device,
                noise_dim=64,
                decoder_type="bp",
                activation=torch.nn.functional.relu,
                dropout=0)
K=15
J=15
batch_graph=[]
for i in range((K+J)):
  train_graph_copy=copy.deepcopy(train_graph)
  batch_graph.append(train_graph_copy)
#batch_graph=[train_graph]*(K+J)
dropout=0
optimizer=torch.optim.Adam(model.parameters(), lr=0.005)
scheduler=optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0, verbose=False)
epoch=2000
loss_accumulate=0
max_test_auc=0
max_test_ap=0
for epc in range(epoch):
    t=time.time()
    model.train()
    
    latent_representation, generated_prob, loss_train, z_proc, rk=step(model, adj_train, adj_label, featureMatrix, train_graph, batch_graph, epc, K, J, weight, norm, dropout, optimizer=optimizer, scheduler=scheduler, device=device)
    #loss_accumulate+=loss_train
    
    #print(z_proc)    
    #print(z_proc.shape)
    #z_proc=z_proc.detach().cpu().numpy()
    
    #roc, avg_precision, precision, recall, fpr, tpr=get_auc(z_proc, pos_array_validation, neg_array_validation, decoder_type="bp")
    roc, avg_precision=get_auc(z_proc, adj, rk,  pos_array_validation, neg_array_validation, decoder_type="bp")
    
    #roc, acc, prc, rcl=get_auc(z_proc, pos_array_train, neg_array_train, decoder_type="bp")
    #print(loss_train.item())
    #print(roc)
    print("Epoch : ", '%d' %(epc+1), "loss=", "%f" %(loss_train.item()),
          "time=", "%f" %(time.time()-t), "roc=", "%f" %(roc), "average_precision", "%f" %(avg_precision),
          #"precision", "%f" %(precision), "recall", "%f" %(recall),
          #"false positive rate", "%f" %(fpr), "true positive rate", "%f" %(tpr))
    )
    #pyplot.plot(fpr, recall, marker='.', label='roc-auc')
    
    
    wandb.log({"loss_train": loss_train})
    wandb.log({"roc-auc": roc})
    wandb.log({"average_precision": avg_precision})
    wandb.watch(model)
    
    '''
    wandb.log({"recall_train" : recall})
    wandb.log({"fpr_train" : fpr})
    wandb.watch(model)
    '''
    del latent_representation
    del generated_prob
    del z_proc
    del roc
    del rk
    del avg_precision
    torch.cuda.empty_cache()
    gc.collect()
    
    if (epc+1) % 20 == 0:
          with torch.no_grad():
            model.eval()
            #generated_prob, mu, sigma, latent_representation, z_proc, epsilon=model(adj_test, featureMatrix, test_graph)
            latent_representation, generated_prob, loss_test, z_proc, rk=step(model, adj_train, adj_label, featureMatrix, train_graph, batch_graph, epc, K, J, weight, norm, dropout, optimizer=None, scheduler=None, device=device)

            #z_proc = z_proc.detach().cpu().numpy()
            roc, avg_pre= get_auc(z_proc, adj, rk, pos_array_test, neg_array_test, decoder_type="bp")
            if roc>max_test_auc:
              max_test_auc=roc
            if avg_pre>max_test_ap:
              max_test_ap=avg_pre
            rslt = "Test ROC score: %f" %(roc)
            rslt2="Test Precision score : %f" %(avg_pre)
            print("\n", rslt, "  ", rslt2, "\n")
                     
            wandb.log({"loss_test": loss_test})
            wandb.log({"roc-auc_test" : roc})
            wandb.log({"ap_test" : avg_pre})
            wandb.watch(model)
            

            del latent_representation
            del generated_prob
            del z_proc
            del roc
            del rk
            #del avg_precision
            torch.cuda.empty_cache()
            gc.collect()
            
#pyplot.show()
print("Max ROC-AUC score : %f" %(max_test_auc))
print("Max AP score : %f" %(max_test_ap))
    #latent_representation, generated_prob, loss_train=step(model, adj_train, epoch, optimizer )

wandb: Currently logged in as: ytrewq271828. Use `wandb login --relogin` to force relogin


^^^^^^^^^^^^^^^^6666
torch.Size([5278])
torch.Size([5278])
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
526
526
****************
2708


/usr/local/lib/python3.7/dist-packages/dgl/heterograph.py:3721: DGLWarning: DGLGraph.adjacency_matrix_scipy is deprecated. Please replace it with:

	DGLGraph.adjacency_matrix(transpose, scipy_fmt="csr").

  'DGLGraph.adjacency_matrix(transpose, scipy_fmt="{}").\n'.format(fmt))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  ../aten/src/ATen/native/Copy.cpp:239.)
  from ipykernel import kernelapp as app


torch.Size([2, 526])
torch.Size([2, 526])
%%%%%%%%%%%%%%%%%%%%%55
2708
Epoch :  1 loss= 1.423288 time= 4.793382 roc= 0.514194 average_precision 0.519463
Epoch :  2 loss= 1.320564 time= 3.138261 roc= 0.502028 average_precision 0.506447
Epoch :  3 loss= 1.346873 time= 3.133195 roc= 0.497272 average_precision 0.499759
Epoch :  4 loss= 1.246844 time= 3.137980 roc= 0.499140 average_precision 0.499347
Epoch :  5 loss= 1.185861 time= 3.156527 roc= 0.493011 average_precision 0.502190
Epoch :  6 loss= 1.188441 time= 3.151460 roc= 0.497162 average_precision 0.501389
Epoch :  7 loss= 1.181154 time= 3.134076 roc= 0.490452 average_precision 0.498635
Epoch :  8 loss= 1.220088 time= 3.130143 roc= 0.496811 average_precision 0.502088
Epoch :  9 loss= 1.100629 time= 3.133632 roc= 0.504651 average_precision 0.505258
Epoch :  10 loss= 1.195333 time= 3.130571 roc= 0.491698 average_precision 0.499901
Epoch :  11 loss= 1.228340 time= 3.191302 roc= 0.500729 average_precision 0.506510
Epoch :  12 loss= 1.09499